In [5]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [2]:
luccrops = pd.read_excel("D:/kerja/asisten riset/code v/step 3/GbGlct/dataset/luc_crop_per_crops_AM2_v1.xlsx")
areaharvd = pd.read_csv("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GFRAC Area HarvestedRegionType_fulll.csv")
country_code = pd.read_excel("D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/ISO-3166-Country-Code_Final.xlsx")

In [6]:
areaharv = areaharvd[areaharvd['time'] == 2020]
# areaharv.loc[:,'time'] = areaharv['time'].replace(1975, 2020)
areaharv.loc[:,'NGFBFC'] = areaharv['NGFBFC'].str.lower()
areaharv.head()

,IMAGE Region Name,NGFBFC,time,value
10,Brazil,maize,2020,2.274070e+07
21,Brazil,oil & palm fruit,2020,2.024777e+05
32,Brazil,other non-food & luxury & spices,2020,4.923515e+06
43,Brazil,other temperate cereals,2020,6.689114e+05
54,Brazil,plant based fibres,2020,1.789001e+06


In [4]:
# 1 a
luccrops_merge = pd.merge(left=luccrops, right=country_code, left_on='country_name', right_on='ISO Country')
luccrops_merge = luccrops_merge[['IMAGE Region Name', 'type', 2020]]
luccrops_melt = luccrops_merge.melt(id_vars=['IMAGE Region Name', 'type'], var_name='time', value_name='value')

lucagri_groupby = luccrops_melt.groupby(['IMAGE Region Name', 'time', 'type']).sum()
lucagri_index = lucagri_groupby.reset_index()
lucagri_index['value'] = lucagri_index['value'] * 1E9
lucagri_index.head()

,IMAGE Region Name,time,type,value
0,Brazil,2020,grass,2.373045e+10
1,Brazil,2020,maize,2.942364e+09
2,Brazil,2020,oil & palm fruit,2.750581e+07
3,Brazil,2020,other non-food & luxury & spices,6.026739e+08
4,Brazil,2020,other temperate cereals,9.029609e+07


In [44]:
# 1 b c
df_copy = areaharv.copy()
for region, time, ngfbfc in zip(df_copy["IMAGE Region Name"].to_list(), df_copy["time"].to_list(), df_copy["NGFBFC"].to_list()):
    try:
        data1 = areaharv[(areaharv['IMAGE Region Name'] == region) & (areaharv['time'] == time) & (areaharv['NGFBFC'] == ngfbfc)]['value'].to_numpy()
        data2 = lucagri_index[(lucagri_index['IMAGE Region Name'] == region) & (lucagri_index['time'] == time) & (lucagri_index['type'] == ngfbfc)]['value'].to_numpy()
        if data1.size == 0:
            data1 = [0]
        if data2.size == 0: 
            data2 = [0]
        mask = df_copy[(df_copy['IMAGE Region Name'] == region) & (df_copy['time'] == time) & (df_copy['NGFBFC'] == ngfbfc)].index
        df_copy.loc[mask, 'CF'] =  data2[0] / data1[0]
    except KeyError or  ZeroDivisionError:
        print(f"Error: {KeyError} for region {region}")

df_copy = df_copy.drop(columns=['value'])
df_copy.head()

,IMAGE Region Name,NGFBFC,time,CF
10,Brazil,maize,2020,129.387574
21,Brazil,oil & palm fruit,2020,135.846127
32,Brazil,other non-food & luxury & spices,2020,122.407232
43,Brazil,other temperate cereals,2020,134.989607
54,Brazil,plant based fibres,2020,115.320221


In [19]:
# 2 a b c
df_copy2 = areaharv.copy()
df_copy2.loc[:, "selisih area harvested"] = np.where(areaharvd[(areaharvd['time'] == 2020)]['value'].to_numpy() > areaharvd[(areaharvd['time'] == 1970)]['value'].to_numpy(),
                        areaharvd[(areaharvd['time'] == 2020)]['value'].to_numpy() - areaharvd[(areaharvd['time'] == 1970)]['value'].to_numpy(), 0)  

df_copy2 = df_copy2[df_copy2["selisih area harvested"] > 0]
df_copy2 = df_copy2.drop(columns=['value'])
df_copy2.head()

,IMAGE Region Name,NGFBFC,time,selisih area harvested
10,Brazil,maize,2020,1.144776e+07
21,Brazil,oil & palm fruit,2020,1.984681e+05
32,Brazil,other non-food & luxury & spices,2020,1.298485e+06
43,Brazil,other temperate cereals,2020,4.121517e+05
87,Brazil,soybeans,2020,4.662921e+07


In [43]:
# 2 d
last_copy = df_copy2.copy()
for region, time, ngfbfc in zip(df_copy2["IMAGE Region Name"].to_list(), df_copy2["time"].to_list(), df_copy2["NGFBFC"].to_list()):
    try:
        data1 = df_copy2[(df_copy2['IMAGE Region Name'] == region) & (df_copy2['time'] == time) & (df_copy2['NGFBFC'] == ngfbfc)]['selisih area harvested'].to_numpy()
        data2 = lucagri_index[(lucagri_index['IMAGE Region Name'] == region) & (lucagri_index['time'] == time) & (lucagri_index['type'] == ngfbfc)]['value'].to_numpy()
        if data1.size == 0:
            data1 = [0]
        if data2.size == 0: 
            data2 = [0]
        mask = last_copy[(last_copy['IMAGE Region Name'] == region) & (last_copy['time'] == time) & (last_copy['NGFBFC'] == ngfbfc)].index
        last_copy.loc[mask, 'CF'] =  data2[0] / data1[0]

    except KeyError:
        print(f"Error: {KeyError} for region {region}")


last_copy = last_copy.drop(columns=["selisih area harvested"])
last_copy.head()

,IMAGE Region Name,NGFBFC,time,CF
10,Brazil,maize,2020,257.025287
21,Brazil,oil & palm fruit,2020,138.590632
32,Brazil,other non-food & luxury & spices,2020,464.136193
43,Brazil,other temperate cereals,2020,219.084587
87,Brazil,soybeans,2020,141.822566


In [53]:
# 3
luc_crops_cf = pd.DataFrame({"IMAGE Region Name": df_copy["IMAGE Region Name"].to_list(), "NGFBFC": df_copy["NGFBFC"].to_list()})
luc_crops_cf = luc_crops_cf.drop_duplicates(subset=["IMAGE Region Name", "NGFBFC"])

for region in last_copy["IMAGE Region Name"].unique():
    for ngfbfc in last_copy["NGFBFC"].unique():
        mask = luc_crops_cf[(luc_crops_cf["NGFBFC"] == ngfbfc) & (luc_crops_cf["IMAGE Region Name"] == region)].index
        data1 = df_copy[(df_copy["NGFBFC"] == ngfbfc) & (df_copy["IMAGE Region Name"] == region)].groupby(['IMAGE Region Name', 'NGFBFC'])['CF'].sum().to_numpy()
        data2 = last_copy[(last_copy["NGFBFC"] == ngfbfc) & (last_copy["IMAGE Region Name"] == region)].groupby(['IMAGE Region Name', 'NGFBFC'])['CF'].sum().to_numpy()
        if data1.size == 0:
            data1 = [0]
        if data2.size == 0: 
            data2 = [0]
        luc_crops_cf.loc[mask, 'average CF'] = (data1[0] + data2[0])/2
luc_crops_cf   

,IMAGE Region Name,NGFBFC,average CF
0,Brazil,maize,193.206431
1,Brazil,oil & palm fruit,137.218379
2,Brazil,other non-food & luxury & spices,293.271713
3,Brazil,other temperate cereals,177.037097
4,Brazil,plant based fibres,57.660111
...,...,...,...
411,W. Europe,tropical cereals,1797.783857
412,W. Europe,tropical oil crops,0.000000
413,W. Europe,tropical roots & tubers,0.000000
414,W. Europe,vegetables & fruits,0.000000
